# 6_Introduction to the Keras Tuner

#### Overview
The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:

1. Model hyperparameters which influence model selection such as the number and width of hidden layers
2. Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

#### Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [3]:
import keras_tuner as kt

#### Download and prepare the dataset
In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the Fashion MNIST dataset.

Load the data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

#### Define the model
When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

- By using a model builder function
- By subclassing the HyperModel class of the Keras Tuner API

You can also use two pre-defined HyperModel classes - HyperXception and HyperResNet for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

#### Instantiate the tuner and perform hypertuning
Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. In this tutorial, you use the Hyperband tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 25s]
val_accuracy: 0.8789166808128357

Best val_accuracy So Far: 0.8920000195503235
Total elapsed time: 00h 04m 38s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



#### Train the model
Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4961 - accuracy: 0.8238 - val_loss: 0.3900 - val_accuracy: 0.8593
Epoch 2/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3730 - accuracy: 0.8638 - val_loss: 0.3874 - val_accuracy: 0.8583
Epoch 3/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3320 - accuracy: 0.8770 - val_loss: 0.3341 - val_accuracy: 0.8802
Epoch 4/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3084 - accuracy: 0.8866 - val_loss: 0.3267 - val_accuracy: 0.8809
Epoch 5/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2873 - accuracy: 0.8937 - val_loss: 0.3328 - val_accuracy: 0.8785
Epoch 6/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2732 - accuracy: 0.8997 - val_loss: 0.3379 - val_accuracy: 0.8806
Epoch 7/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2609 - accuracy: 0.9026 - val_loss: 0.3402 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/19
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4935 - accuracy: 0.8248 - val_loss: 0.3869 - val_accuracy: 0.8620
Epoch 2/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3707 - accuracy: 0.8659 - val_loss: 0.3652 - val_accuracy: 0.8677
Epoch 3/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3309 - accuracy: 0.8776 - val_loss: 0.3508 - val_accuracy: 0.8716
Epoch 4/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3048 - accuracy: 0.8867 - val_loss: 0.3269 - val_accuracy: 0.8800
Epoch 5/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2873 - accuracy: 0.8934 - val_loss: 0.3334 - val_accuracy: 0.8799
Epoch 6/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2684 - accuracy: 0.9006 - val_loss: 0.3105 - val_accuracy: 0.8878
Epoch 7/19
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2558 - accuracy: 0.9043 - val_loss: 0.3202 - val_accuracy:

To finish this tutorial, evaluate the hypermodel on the test data.

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 849us/step - loss: 0.4032 - accuracy: 0.8837
[test loss, test accuracy]: [0.40317755937576294, 0.8837000131607056]


The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.